In [1]:
import torch
import torchvision
import torch.nn as nn
from utils import compute_iou
from data.cbis_ddsm import CBIS_DDSM, get_loaders
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from collections import defaultdict
from tqdm import tqdm




In [2]:
# Check if gpu is available

if not torch.cuda.is_available():
  raise Exception("GPU not availalbe. CPU training will be too slow.")

print("device name", torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device', device)

CUDA_LAUNCH_BLOCKING=1

device name NVIDIA GeForce GTX 1080 Ti
device cuda


In [3]:
def print_metrics(metrics, epoch_samples):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("Metrics: {}".format(", ".join(outputs)))

In [4]:
def calc_metrics(outputs, target, metrics, bce_weight=0.5):
    # bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = torch.sigmoid(outputs)
    # dice = dice_loss(pred, target)

    # loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    # metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    
    
    

    # return loss

In [5]:
def test_model(model, testloader):
    
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        
        for i, batch in enumerate(tqdm(testloader)):

            inputs, target = batch['image'], batch['target']
            # mask = mask.mean(1,keepdim=True)  #transform from [batch_size, 3, size, size] to [batch,size, 1, size, size]
            inputs = inputs.to(device)
            target = target.to(device)
            
            outputs = model(inputs)
            
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    print(f'Test Acc: {100 * correct // total} %')

In [6]:
### Parameters

num_epochs = 10

num_class = 2 #binary problem  
# model = ResNetUNet(num_class).to(device)
model = torchvision.models.resnet18(pretrained=True)

# Freeze model weights
# for param in model.parameters():
#     param.requires_grad = False

num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, num_class)
model = model.to(device)

criterion = nn.CrossEntropyLoss()


# freeze backbone layers
# for l in model.base_layers:
#     for param in l.parameters():
#         param.requires_grad = False
        
        
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

#dataloader parameters

args={
    'data_path': "/datasets/mamografia/CBIS-DDSM_organized/images/preprocessed",
    "batch_size": 64,
    "size1": 270,
    "size": 256
}


train_loader, test_loader = get_loaders(args)

In [ ]:


## Training Loop
best_loss = 1e10

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    since = time.time()
    model.train()
    
    metrics = defaultdict(float)
    epoch_samples = 0
    total = 0
    correct = 0

    print(f"Epoch {epoch} - Training...\n")
    # for inputs, labels in dataloaders[phase]:
    for i, batch in enumerate(tqdm(train_loader)):
        
        inputs, target = batch['image'], batch['target']
        
        # mask = mask.mean(1,keepdim=True)  #transform from [batch_size, 3, size, size] to [batch,size, 1, size, size]
        
        inputs = inputs.to(device)
        target = target.to(device)
        # mask = mask.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        # with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)
        
        # loss = calc_loss(outputs, mask, metrics, bce_weight=1)
        
        loss = criterion(outputs, target)

        loss.backward()
        optimizer.step()

        # statistics
        epoch_samples += inputs.size(0)
        
        # print statistics
        running_loss = loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
        # running_loss = 0.0

    scheduler.step()
    print(f'Train Acc: {100 * correct // total} %')
    
    #test
    print("Testing...")
    test_model(model, test_loader)
    


    time_elapsed = time.time() - since
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

# torch.save(model.state_dict(), "trained_model.pth")



Epoch 0/9
----------
Epoch 0 - Training...



  5%|▌         | 1/20 [00:57<18:15, 57.66s/it]

[1,     1] loss: 0.853


 15%|█▌        | 3/20 [00:58<03:42, 13.09s/it]

[1,     2] loss: 0.858
[1,     3] loss: 0.791


...

[4,     1] loss: 0.678


 10%|█         | 2/20 [00:59<07:24, 24.67s/it]

[4,     2] loss: 0.641


 20%|██        | 4/20 [01:01<02:20,  8.76s/it]

[4,     3] loss: 0.649
[4,     4] loss: 0.677
